In [ ]:
import xarray as xr
import pathlib as pl
import pandas as pd
import pywatershed
import os
import dask
import numpy as np


In [ ]:
all_models = ['01473000', '05431486','09112500','14015000']
var_output_files = ['seg_outflow.nc','hru_actet.nc', 'snowcov_area.nc', 'recharge.nc', 'soil_rechr.nc' ]
# These variables will be post-processed to match with observations (targets)

In [ ]:
os.chdir("..")
base_dir = os.path.abspath(os.curdir)
print(base_dir)

In [ ]:
cdir = f'{base_dir}/NHM_extractions/20230110_pois_haj/'
print(cdir)

In [ ]:
odir = pl.Path(f'{cdir}/01473000/output/')

In [ ]:
NHM_output = xr.open_mfdataset(odir.glob('*.nc'))

In [ ]:
########### Useful code
#indat
#df.index.dtypes
#xr.__version__

### Post Process "hru_actet" to compare to target
#### Get and check the daily data

In [ ]:
#NHM_output.hru_actet

#### Post-process daily output to match observation targets

In [ ]:
#Creates a dataframe time series of monthly mean values
actet_monthly = NHM_output.hru_actet.resample(time = 'm').mean() 
actet_monthly_df = actet_monthly.to_dataframe()# makes a dataframe for plotting and writing out
actet_monthly_df_r = np.ravel(actet_monthly_df, order = 'C')# flattens the 2D array to a 1D array--just playing 

# Creates a dataframe time series of mean of monthly means
actet_mean_monthly = actet_monthly.groupby('time.month').mean()

# Creates a dataframe time series of min of monthly means
actet_mean_monthly_min = actet_monthly.groupby('time.month').min() 

# Creates a dataframe time series of max of monthly means
actet_mean_monthly_max = actet_monthly.groupby('time.month').max() 


#### Check out results to verify

In [ ]:
#actet_monthly_df_r[0:20]
#actet_monthly_df
#actet_monthly_df.xs(5621, level=1).plot()

### Post Process "recharge" to compare to target
#### Get and check the daily data

In [ ]:
#NHM_output.recharge

#### Post-process daily output to match observation targets

In [ ]:
#Creates a dataframe time series of monthly mean values
recharge_annual = NHM_output.recharge.resample(time = 'Y').mean() 

recharge_annual_df = recharge_annual.to_dataframe()# makes a dataframe for plotting and writing out
recharge_annual_df_r = np.ravel(recharge_annual_df, order = 'C')# flattens the 2D array to a 1D array--just playing 



In [ ]:
recharge_annual

#### Check out results to verify

In [ ]:
#recharge_annual_df_r[0:20]
#recharge_annual_df
#recharge_annual_df.xs(5621, level=1).plot()

### Post Process "soil_rechr" to compare to target
#### Get and check the daily data

In [ ]:
NHM_output.soil_rechr

In [ ]:
#Creates a dataframe time series of monthly mean values
soil_rechr_monthly = NHM_output.soil_rechr.resample(time = 'm').mean() 
soil_rechr_monthly_df = soil_rechr_monthly.to_dataframe()# makes a dataframe for plotting and writing out
soil_rechr_monthly_df_r = np.ravel(soil_rechr_monthly_df, order = 'C')# flattens the 2D array to a 1D array--just playing 

#Creates a dataframe time series of annual values
soil_rechr_annual = NHM_output.soil_rechr.resample(time = 'Y').mean() 
soil_rechr_annual_df = soil_rechr_annual.to_dataframe()# makes a dataframe for plotting and writing out
soil_rechr_annual_df_r = np.ravel(soil_rechr_annual_df, order = 'C')# flattens the 2D array to a 1D array--just playing 


#### Check out results to verify

In [ ]:
#soil_rechr_monthly_df_r[0:20]
#soil_rechr_monthly_df
#soil_rechr_monthly_df.xs(5621, level=1).plot()

### Post Process "snowcov_area" to compare to target
#### Get and check the daily data

In [ ]:
NHM_output.snowcov_area

In [ ]:
#Creates a dataframe time series of daily values
snowcov_areas_daily = NHM_output.snowcov_area
snowcov_areas_daily_df = snowcov_areas_daily.to_dataframe()# makes a dataframe for plotting and writing out
snowcov_areas_daily_df_r = np.ravel(snowcov_areas_daily_df, order = 'C')# flattens the 2D array to a 1D array--just playing 

In [ ]:
snowcov_areas_daily_df_r[0:20]
#snowcov_areas_daily_df
#snowcov_areas_daily_df.xs(5621, level=1).plot()

In [ ]:
snowcov_areas_daily_df

In [ ]:
#
## example of making index from multi-level dataframe
#

# MONTHLY
varname = 'aet_min'
inds = actet_monthly_df.index.map(lambda x: f'{varname}_mon:{x[0].year}_{x[0].month}:{x[1]}')
print(inds[0])
# MEAN MONTHLY
varname = 'aet_min'
inds = actet_mean_monthly.to_dataframe().index.map(lambda x: f'{varname}_mean_mon:{x[0]}:{x[1]}')
print(inds[0])

In [ ]:
# create proper indices and ravel values as needs and write to consolidated file
with open(odir / 'all_mod_output.dat', 'w') as ofp:
    
    # aet min monthly
    varname = 'aet_min'
    inds = actet_monthly_df.index.map(lambda x: f'{varname}_{x[0].year}_{x[0].month}:{x[1]}')
    varvals = actet_monthly_df.to_numpy().ravel()
    [ofp.write(f'{i}  {j}\n') for i,j in zip(inds,varvals)]

### Post Process "seg_outflow" to compare to target
#### Get and check the daily data

In [ ]:
NHM_output.hru_streamflow_out